In [44]:
from google.genai import types

# Define a function that the model can call to control smart lights
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}


In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def call_gemini(user_prompt=None, system_prompt=None, functions=None):
    """
    Call the Gemini API with optional user prompt, system prompt, and functions.

    Args:
        user_prompt (str): The user prompt to include in the API call.
        system_prompt (str): The system prompt to include in the API call.
        functions (list): A list of function declarations to include in the API call.

    Returns:
        dict: The response from the Gemini API.
    """
    # Load API key and URL
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    gemini_url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
    tools = types.Tool(function_declarations=[functions])
    config = types.GenerateContentConfig(tools=[tools])
    # Define headers
    headers = {
        "Content-Type": "application/json"
    }

    contents = []

    # Add system and user prompts to the payload
    if system_prompt or user_prompt:
        prompt = ""
        if system_prompt:
            contents.append({
                types.Content(
                    role='system',
                    parts=[types.Part(text=system_prompt)]
                )
            })
        if user_prompt:
            contents.append({
                types.Content(
                    role='user',
                    parts=[types.Part(text=user_prompt)]
                )
            })
    # Send request with function declarations
    response = client.models.generate_content(
        model="gemini-2.0-flash", config=config, contents=contents
    )
    # Add functions to the payload if provided
    if functions:
        payload["tools"] = [
            {
                "functionDeclarations": functions
            }
        ]

    # Debugging: Print the payload
    print(f"Payload: {json.dumps(payload, indent=2)}")

    # Make the POST request
    try:
        response = requests.post(
            f"{gemini_url}?key={gemini_api_key}",
            headers=headers,
            data=json.dumps(payload)
        )
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()  # Return the parsed JSON response
    except requests.exceptions.RequestException as e:
        print(f"Error while calling Gemini API: {e}")
        return None

In [51]:
call_gemini(
    user_prompt="Set the light to 50% brightness and warm color temperature.",
    system_prompt="You are a smart home assistant.",
    functions=[set_light_values_declaration]
)
set_light_values_declaration

Payload: {
  "contents": [
    {
      "parts": [
        {
          "text": "You are a smart home assistant.\nSet the light to 50% brightness and warm color temperature."
        }
      ]
    }
  ],
  "tools": [
    {
      "functionDeclarations": [
        {
          "name": "set_light_values",
          "description": "Sets the brightness and color temperature of a light.",
          "parameters": {
            "type": "object",
            "properties": {
              "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness"
              },
              "color_temp": {
                "type": "string",
                "enum": [
                  "daylight",
                  "cool",
                  "warm"
                ],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`."
              }
            },
            "re

{'name': 'set_light_values',
 'description': 'Sets the brightness and color temperature of a light.',
 'parameters': {'type': 'object',
  'properties': {'brightness': {'type': 'integer',
    'description': 'Light level from 0 to 100. Zero is off and 100 is full brightness'},
   'color_temp': {'type': 'string',
    'enum': ['daylight', 'cool', 'warm'],
    'description': 'Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.'}},
  'required': ['brightness', 'color_temp']}}